In [1]:
# Import the requests library
import requests
# Import the API key.
from config import weather_api_key

In [2]:
# Starting URL for Weather Map API call.
url = "https://api.openweathermap.org/data/2.5/weather?units=imperial&appid=" + weather_api_key


In [3]:
# Create an endpoint URL for a city.
city="Houston"
query_url = url + "&q=" + city
city_weather = requests.get(query_url)
city_weather

<Response [200]>

In [4]:

city_weather = requests.get(query_url)
city_weather_json = city_weather.json()
city_weather_json.keys()
# dict_keys = ({'coord', 'weather', 'base', 'main', 'visibility', 'wind', 'rain'})
city_weather_json.get('main')

{'temp': 94.15,
 'feels_like': 106.75,
 'temp_min': 90.91,
 'temp_max': 98.31,
 'pressure': 1015,
 'humidity': 58}

In [5]:
city_weather = requests.get(query_url)
city_weather

<Response [200]>

In [6]:
city_weather.status_code

200

In [7]:
city_weather.json()

# With the JSON method, it is a lot easier to see the overall structure of the data,
# which will make it easier to retrieve data such as temperature and humidity.

{'coord': {'lon': -95.3633, 'lat': 29.7633},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': 94.15,
  'feels_like': 106.75,
  'temp_min': 90.91,
  'temp_max': 98.31,
  'pressure': 1015,
  'humidity': 58},
 'visibility': 10000,
 'wind': {'speed': 0, 'deg': 0},
 'clouds': {'all': 0},
 'dt': 1693068129,
 'sys': {'type': 1,
  'id': 4850,
  'country': 'US',
  'sunrise': 1693050927,
  'sunset': 1693097497},
 'timezone': -18000,
 'id': 4699066,
 'name': 'Houston',
 'cod': 200}

In [8]:
city_weather.status_code

200

# Handle Error Requests

In [9]:
# When we submit a get() request for the city_weather, we want to make sure that
# we get a valid response, i.e. 200, before we retrieve any data.

# Create an endpoint URL for a city.
city = "Houston"
city_url = url + "&q=" + city

city_weather = requests.get(city_url)

if city_weather.status_code == 200:
    print(f'City Weather Found.')
else:
    print(f'City Weather NOT found.')

# If the status_code is something other than 200, JSON will always be returned
# in the request. We can determine if the response was successful by checking the 
# status_code, clicking on the url, or retrieving specific information from the JSON data.



City Weather Found.


In [10]:
# Mod 6.2.5 parse a Response from an API

# Start mining the JSON file to retrieve specific weather data for each city and add it to 
# a DataFrame.

# For each city in our lat_lngs, we need to retrieve the following data and add it to a DataFrame:
#   --> city, country, and date
#   --> latitude and longitude
#   --> Maximun Temperature
#   --> humidity
#   --> cloudiness
#   --> wind speed
# The final data should look like: (see page 27 of WeatherPy notes)

## The final data should look like: (see page 27 of WeatherPy notes)

In [11]:
houston_data = city_weather.json()
houston_data

{'coord': {'lon': -95.3633, 'lat': 29.7633},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': 94.15,
  'feels_like': 106.75,
  'temp_min': 90.91,
  'temp_max': 98.31,
  'pressure': 1015,
  'humidity': 58},
 'visibility': 10000,
 'wind': {'speed': 0, 'deg': 0},
 'clouds': {'all': 0},
 'dt': 1693068129,
 'sys': {'type': 1,
  'id': 4850,
  'country': 'US',
  'sunrise': 1693050927,
  'sunset': 1693097497},
 'timezone': -18000,
 'id': 4699066,
 'name': 'Houston',
 'cod': 200}

In [12]:
lat = houston_data['coord']['lat']
lng = houston_data['coord']['lon']
max_temp = houston_data['main']['temp_max']
humidity = houston_data['main']['humidity']
clouds = houston_data['clouds']['all']
wind = houston_data['wind']['speed']
print(lat, lng, max_temp, humidity, clouds, wind)

29.7633 -95.3633 98.31 58 0 0


### Convert the timedate stamp

In [13]:
date = houston_data['dt']

# This format is called "coordinated universal time (UTC) or Greenwich Mean Time (GMT). 
# If we want to convert the timestamp to the International Organization of Standardization (ISO)
# format, or YYYY-MM-DD-HH-MM-SS, we need to use the python "datetime" modules.

# Let's convert the date from the Houston weather data in JSON format to the ISO format.

# Import the datetime module from the datetime library
from datetime import datetime

# Get the data from the JSON file
date = houston_data['dt']

# Convert the UTC date to a date format with YYYY-MM-DD-HH-MM-SS
datetime.utcfromtimestamp(date)

# We can convert this datetime format to 2023-08-26-15 15:51:47 using the 
# python string format strftime() method and adding how we want the string
# to look inside the parentheses. 

# In our case, we would use strftime('%y-%m-%d %H:%M:%S')
datetime.utcfromtimestamp(date).strftime('%y-%m-%d %H:%M:%S')

# NOTE: For more information about the 'datetime' library amd the strftime() method, 
# see the documentation: 
# --> datetime
# --> strftime



'23-08-26 16:42:09'

## Mod 6.2.6 Get the City Weather Data - retrieving the weather data for 500+ cities.

### Let's use pseudocode first:
#### 1. Import our dependencies and initialize counter and an empty list that will hold the weather data.
#### 2. Loop through the cities list.
#### 3. Group the cities in sets of 50 to log the process as we find the weather data for each city.
#####     Two counters will need to be here: one to log the city count from 1 to 50, and another for the sets.
#### 4. Build the city_url or endpoint for each city.
#### 5. Log the url and record and set numbers.
#### 6. Make an api request for each city.
#### 7. Parse the JSON weather data for the following
##### ---> city, country, and date
##### ---> latitude and longitude
##### ---> maximum temperature
##### ---> humidity
##### ---> cloudiness
##### ---> wind speed
#### 8. Add the data to a list in a dictionary format and then convert it to a DataFrame.